In [2]:
!pip install gradio transformers langchain faiss-cpu sentence-transformers matplotlib langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing 

In [9]:
import os
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import io
import tempfile
from sklearn.metrics.pairwise import cosine_similarity
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.document_loaders import WikipediaLoader

summarizer = pipeline("text-generation", model="UnfilteredAI/UNfilteredAI-1B")

classification_model_name = "andreas122001/bloomz-1b7-mixed-detector"
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"

tokenizer = AutoTokenizer.from_pretrained(classification_model_name)
classification_model = AutoModelForSequenceClassification.from_pretrained(classification_model_name)
text_classifier = pipeline("text-classification", model=classification_model, tokenizer=tokenizer)

embedder = SentenceTransformer(embedding_model_name)

# Knowledge Base with Grooming Patterns
def generate_grooming_knowledge_base(file_path):
    """Generate a realistic knowledge base of grooming behaviors."""
    if not os.path.exists(file_path):
        with open(file_path, "w") as f:
            f.write("\n".join([
    # Compliments and Trust-Building
    "You’re so mature for your age.",
    "I feel like I can really trust you.",
    "You seem really smart and independent. I bet most people don’t get you like I do.",
    "We have such a special connection. I don’t feel this way about anyone else.",
    "I’ve never met someone your age who’s so cool.",
    "You’re really special to me.",
    "I’ve never felt this way about anyone before.",
    "You can tell me anything. I won’t judge you.",
    "You’re the only person who really gets me.",
    "You make me feel so comfortable around you.",
    "I’m so glad I met you. I feel like I can share everything with you.",

    # Secrecy and Isolation
    "Don’t tell anyone we’re talking. They wouldn’t understand.",
    "This is our little secret.",
    "We don’t need anyone else; we can just talk to each other.",
    "You can trust me. Adults just don’t get how special our connection is.",
    "Let’s keep this between us, okay?",
    "Your parents don’t need to know about this.",
    "I’d get in trouble if anyone found out we were talking.",
    "Your parents won’t understand our friendship.",
    "You can talk to me about anything, just keep it between us.",
    "It’s our secret. Don’t let anyone find out what we talk about.",
    "You can’t tell your friends or parents. It’s not something they would understand.",
    "Our connection is special—no one else can understand it.",

    # Manipulating the Victim’s Age
    "You’re so much more mature than kids your age.",
    "I can’t believe you’re only [age]! You seem so much older.",
    "You’re too smart to hang out with kids your age.",
    "You’re so much more mature than your parents think.",
    "You’re different from other kids—much more grown up.",
    "People your age don’t understand things the way you do.",
    "I think you’re beyond your years. You’re really something special.",

    # Testing Boundaries
    "What’s your favorite thing to do when no one’s watching?",
    "Do you feel comfortable talking about [sensitive topic]?",
    "Would you ever do something like [suggestive action]?",
    "Would you like to share more personal things with me? I won’t judge you.",
    "Are you comfortable with me asking you more personal questions?",
    "You don’t have to answer if you’re not comfortable, but can I ask you something personal?",
    "How far would you go if we met in person? Do you think about things like that?",
    "What kind of stuff do you fantasize about?",
    "I’ve never told anyone this before, but I think you’re really special.",

    # Progression Toward Meeting
    "Maybe we could meet someday—just as friends.",
    "I’d love to meet you in person someday.",
    "What do you do when you’re alone?",
    "Do you ever hang out with older people?",
    "What’s the most personal thing you’ve ever told someone?",
    "Would you like to do something special when we meet?",
    "How about we meet somewhere private and spend time alone?",
    "Would you ever want to stay over at my place? We could just hang out.",
    "If we met in person, would you feel comfortable being close to me?",
    "I’m really looking forward to seeing you. I think we’d get along even better in person.",

    # Sexual and Inappropriate Requests
    "Would you like to be more than just friends?",
    "Do you want to take our relationship to the next level?",
    "I can’t stop thinking about you. Do you feel the same way?",
    "What if we got together in a more intimate way?",
    "Would you like to send me a picture of yourself?",
    "Can you tell me what you look like in detail? I’m curious.",
    "You’re so cute. I wonder what it would be like if we were more than friends.",
    "Maybe we could kiss someday—just as a joke, right?",
    "I’d love to get closer to you—how about a hug or something more private?",
    "What would you wear if we met up alone?",
    "Let’s keep this between us, but would you like to talk about more intimate things?",
    "Would you be okay with us getting closer physically? No one has to know.",
    "If you ever feel uncomfortable, let me know, but I think we’d be great together, physically.",
    "I think we could be really close if we spent more time together, don’t you?",
    "Do you think you could ever love someone like me?",
    "Would you like to experience something new with me, just us?",
    "You can trust me—there’s nothing wrong with exploring these things together.",

    # Coercive Requests and Manipulation
    "It’s okay to explore your feelings, I promise I won’t tell anyone.",
    "If you ever change your mind, I’ll still be here for you.",
    "I know you want to be close to me, and I think it’s something special we have.",
    "We could keep it secret, but I’d love to see what happens if we meet.",
    "Do you want to feel special? You are to me, and we could make it even better.",
    "If I could be with you in person, I’d make you feel amazing.",
    "I can help you discover things you might not have thought about before.",
    "You don’t need to be shy around me. I think we’d have a great time together.",
    "I know you’re curious, and I can show you things that will make you feel good.",

    # Parental Manipulation and Isolation
    "Do you ever feel like your parents don’t trust you?",
    "Maybe we could meet someday—just as friends.",
    "Do you think your parents would be okay with us hanging out?",
    "I’ll keep it secret if you’re nervous about meeting.",
    "I bet your parents wouldn’t understand how close we are. We don’t need them to know.",
    "You don’t have to tell your parents about us. It’ll be our little secret.",
    "Your parents might overreact if they knew what we talk about. Let’s keep it private.",

    # Example Conversations
    "Example 1: Person A compliments Person B and asks their age, emphasizing their maturity.",
    "Example 2: Person A builds trust and requests secrecy by saying, 'Don’t tell anyone we’re talking.'",
    "Example 3: Person A: 'What kind of things do you like doing after school?'",
    "Example 4: Person A asks about Person B’s parents and suggests keeping things private.",
    "Example 5: Person A flirts with Person B and emphasizes their maturity.",
    "Example 6: Person A reinforces emotional isolation by stating, 'People your age don’t get you.'",
    "Example 7: Person A suggests secrecy to avoid parental involvement.",
    "Example 8: Person A suggests or requests more intimate interactions, indicating a shift toward sexual conversations.",
    "Example 9: Person A manipulates the victim into believing they are special and deserving of intimacy.",
    "Example 10: Person A pushes boundaries by offering secrecy in exchange for physical intimacy.",
    "Example 11: Person A pressures Person B into thinking that physical closeness is necessary to prove their bond.",
            ]))
        print(f"Generated grooming knowledge base at: {file_path}")

def expand_knowledge_base_with_rag(existing_knowledge_base_file, topic="child grooming"):
    """
    Use RAG to dynamically retrieve patterns and expand the knowledge base.
    """
    loader = WikipediaLoader(query=topic, lang="en")
    documents = loader.load()
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    vector_store = FAISS.from_documents(documents, embeddings)

    retriever = vector_store.as_retriever(search_kwargs={"k": 5})
    prompt = PromptTemplate(
        template="Extract grooming-related patterns from the following text: {context}",
        input_variables=["context"]
    )
    qa_chain = RetrievalQA.from_chain_type(
        llm=text_classifier,
        retriever=retriever,
        prompt=prompt
    )

    retrieved_patterns = []
    for doc in documents:
        response = qa_chain.run({"context": doc.page_content})
        retrieved_patterns.append(response)

    with open(existing_knowledge_base_file, "r") as f:
        existing_patterns = set(f.read().splitlines())

    new_patterns = set(retrieved_patterns)
    combined_patterns = existing_patterns.union(new_patterns)

    with open(existing_knowledge_base_file, "w") as f:
        f.write("\n".join(combined_patterns))

    print(f"Knowledge base expanded with {len(new_patterns)} new patterns.")

# Loading Knowledge Base into FAISS
def create_knowledge_base(data_path):
    """Create or load a FAISS index for the knowledge base."""
    if not os.path.exists(data_path):
        raise FileNotFoundError(f"The file {data_path} does not exist. Please provide a valid file.")

    loader = TextLoader(data_path)
    documents = loader.load()
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    vector_store = FAISS.from_documents(documents, embeddings)
    return vector_store

knowledge_base_file = "expanded_grooming_patterns.txt"
generate_grooming_knowledge_base(knowledge_base_file)
knowledge_base = create_knowledge_base(knowledge_base_file)

# Retrieve Context
def retrieve_context(chat_log):
    """Retrieve related examples from the knowledge base."""
    retriever = knowledge_base.as_retriever(search_kwargs={"k": 5})
    retrieved_docs = retriever.invoke(chat_log)
    return [doc.page_content for doc in retrieved_docs]

# Match Behavior Patterns
def match_behavior_patterns(chat_log, patterns, embedder):
    """Match behavioral patterns in the chat log."""
    pattern_embeddings = embedder.encode(patterns, batch_size=8)
    chat_embedding = embedder.encode([chat_log])[0]
    similarities = cosine_similarity([chat_embedding], pattern_embeddings)[0]
    matches = [patterns[i] for i, score in enumerate(similarities) if score > 0.5]
    return len(matches), matches

# Generate Human-Readable Explanations
def generate_explanations(matched_patterns, risk_level):
    """Generate a concise explanation with actionable recommendations based on risk level."""
    if not matched_patterns:
        return (
            "The conversation does not exhibit concerning patterns typically associated with grooming behaviors. "
            "It appears casual and appropriate. However, continued monitoring of interactions is recommended."
        )

    pattern_summary = f"{len(matched_patterns)} specific behavioral patterns were detected that may indicate grooming risk."

    if risk_level > 8:
        recommendation = (
            "The risk level is critically high. Immediate action is required. Consider involving legal authorities "
            "or child protection services to ensure safety."
        )
    elif 5 < risk_level <= 8:
        recommendation = (
            "The risk level is moderate to high. Serious monitoring of future interactions is advised, "
            "and discussions with a trusted adult or guardian may be necessary."
        )
    else:
        recommendation = (
            "The risk level is low to moderate. While immediate intervention may not be necessary, "
            "continued observation and maintaining open communication with the involved individual is crucial."
        )

    input_text = (
        f"The conversation exhibits potential grooming behaviors. {pattern_summary} "
        f"The risk level is assessed at {risk_level:.2f}/10. Provide a concise explanation of the risks and "
        f"recommendations for handling this scenario."
    )

    summary = summarizer(
        input_text,
        max_new_tokens=100,
        num_return_sequences=1,
        do_sample=True,
        truncation=True
    )[0]['generated_text']

    return f"{summary}\n\nRecommendations: {recommendation}"


# Risk Scoring with Dynamic Weights
def calculate_risk(chat_log, retrieved_context, embedder):
    classification_response = text_classifier(chat_log, truncation=True, max_length=512)
    classification_label = classification_response[0]['label']

    pattern_score, matched_patterns = match_behavior_patterns(chat_log, retrieved_context, embedder)

    fallback_cues = ["how old", "don’t tell", "secret", "just us", "you’re mature", "parents"]
    cue_score = sum(1 for cue in fallback_cues if cue.lower() in chat_log.lower())

    secrecy_weight = 3 if any(cue in chat_log.lower() for cue in ["don’t tell", "secret"]) else 0

    risk_score = pattern_score * 2 + cue_score * 1.5 + secrecy_weight
    risk_score = min(risk_score, 10)

    explanation = generate_explanations(matched_patterns, risk_score)
    return risk_score, matched_patterns, explanation

# Visualization
def create_visualization(risk_score):
    img_buffer = io.BytesIO()
    plt.figure(figsize=(5, 3))

    color = 'green' if risk_score <= 3 else 'yellow' if risk_score <= 6 else 'red'

    plt.bar(["Risk Score"], [risk_score], color=color)
    plt.ylim(0, 10)
    plt.title("Risk Score Visualization")
    plt.savefig(img_buffer, format='png')
    plt.close()

    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
    with open(temp_file.name, "wb") as f:
        f.write(img_buffer.getvalue())
    return temp_file.name

# Analyze Chat Function
def analyze_chat(chat_log):
    retrieved_context = retrieve_context(chat_log)
    risk_score, matched_patterns, explanation = calculate_risk(chat_log, retrieved_context, embedder)
    visualization = create_visualization(risk_score)
    return f"Risk Score: {risk_score:.2f}/10", visualization, explanation

# Gradio Interface
with gr.Blocks() as interface:
    gr.Markdown("## Child Grooming Detection System")

    with gr.Row():
        chat_log_input = gr.Textbox(
            label="Chat Log",
            lines=10,
            placeholder="Paste a conversation or chat log here...",
        )

    with gr.Row():
        risk_output = gr.Textbox(label="Risk Score")
        visualization_output = gr.Image(label="Risk Visualization")

    explanation_output = gr.Textbox(
        label="AI Insights",
        lines=8,
        interactive=False,
        placeholder="Detailed analysis will appear here...",
    )

    with gr.Row():
        analyze_button = gr.Button("Analyze")

    analyze_button.click(
        analyze_chat,
        inputs=[chat_log_input],
        outputs=[risk_output, visualization_output, explanation_output],
    )

interface.launch()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c16febe410d791683a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
